# Pupil-Core Visualization Dashboard

## Preparation of environment

### Install dependencies

In [1]:
!pip install notebook --upgrade
!pip install pandas
!pip install numpy
!pip install ipywidgets
!pip install ipython
!pip install opencv-python
!pip install jupyter_contrib_nbextensions
!pip install --upgrade jupyter_contrib_nbextensions



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


### Import libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.stats import gaussian_kde
import ipywidgets as widgets
from ipywidgets import Layout



### Config:
From here you can configure certain aspects of visualization and estimations

In [3]:
WIDTH = 480
HEIGHT = 320
VIDEO_WIDTH, VIDEO_HEIGHT = 1280, 720
IMG_SIZE = (1200, 1200)
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.5
TEXT_COLOR = (0, 0, 0)  # Black color
TEXT_THICKNESS = 1
TEXT_LINE_SPACING = 20
FOV_THICKNESS = 2
FOV_COLOR = (1,0,0)
MARKER_THICKNESS=2
MARKER_VISIBLE_COLOR = (0,0,1)
MARKER_NOT_VISIBLE_COLOR = (0,1,1)
GAZE_CIRCLE_COLOR = (0,1,0)
GAZE_CIRCLE_RADIUS = 15
MARKER_ENLARGEMENT_RATE = 2
EYE_DIAMETER_CIRCLE_THICKNESS = 2
EYE_DIAMETER_NORMALIZING_FACTOR = 50
EYE_DILATION_COLOR = (1,0,0)
BLINK_LIST_LENGTH = 3
NUMBER_OF_PAST_POSES_TO_VISUALIZE = 10  # number of past poses to store and visualize
GAZE_KERNEL_FOR_HEATMAP = 5
FPS = 24
pd.options.mode.chained_assignment = None  # default='warn'


### Configure Filepaths:

This section must be configured correctly to address the files that are to be used in the process

In [4]:
headpose_tracker_poses_path = "resources/001/head_pose_tracker_poses.csv"
marker_detections_path = "resources/001/marker_detections.csv"
video_path = "resources/001/world.mp4"
world_timestamps_path = "resources/001/world_timestamps.csv"
pupil_positions_path = "resources/001/pupil_positions.csv"
gaze_positions_path = "resources/001/gaze_positions.csv"
blinks_path = "resources/001/blinks.csv"

assert os.path.exists(headpose_tracker_poses_path)
assert os.path.exists(marker_detections_path)
assert os.path.exists(video_path)
assert os.path.exists(world_timestamps_path)
assert os.path.exists(pupil_positions_path)
assert os.path.exists(gaze_positions_path)
assert os.path.exists(blinks_path)

### Import Dependencies

In [5]:
from file_utils import get_eye_diameters
from file_utils import get_gaze_positions
from file_utils import get_blinks
from file_utils import get_head_poses
from file_utils import get_marker_positions
from file_utils import get_frames

from utils import get_marker_count_per_frame
from utils import get_continous_blinks_list
from utils import estimate_gaze_pos
from utils import get_translated_view_rectangle
from utils import get_adjusted_normalized_markers



## Implementation

### Get Data:
In this section, we are retrieving the data from csv files, and process them to estimate and visualize the details.

In [6]:
pupil_dilations_df = get_eye_diameters(pupil_positions_path)
gaze_positions_df = get_gaze_positions(gaze_positions_path)
blinks = get_blinks(blinks_path)
head_poses_df = get_head_poses(headpose_tracker_poses_path, world_timestamps_path)
marker_positions = get_marker_positions(marker_detections_path)

marker_counts_per_frame = get_marker_count_per_frame(marker_positions)
frames, number_of_frames, frame_width, frame_heigth, fps = get_frames(video_path)
continous_blinks_list = get_continous_blinks_list(blinks.values.tolist(), number_of_frames)

### Additional functions:
This section stores a function for only to be used in this visualization dashboard.

In [7]:
def get_heatmap_gaze_positions(gaze_positions, head_poses, frame_index, range_of_frames):
    heatmap_gaze_positions = []
    start_index = max(0, frame_index - range_of_frames)
    end_index = max(frame_index,start_index+3)
    for i in range(start_index,end_index):
        pitch, yaw, roll = head_poses.iloc[i]
        fov_center, fov_rectangle = get_translated_view_rectangle(pitch, -roll, yaw)

        heatmap_gaze_positions.append(estimate_gaze_pos(fov_center,gaze_positions.iloc[i],roll))
    return heatmap_gaze_positions


### Plot Functions:
These are seperate plotting functions to visualize certain aspects of the data we have

In [8]:
def plot_cumulatively_eye_diameters(show_both_eyes = False, show_mean = True):
    plt.clf()  # Clear the current figure

    df = pupil_dilations_df
    # Calculate the mean of eye diameters
    df['mean_diameter'] = pupil_dilations_df.mean(axis=1)

    # Convert frame index to seconds
    df['time'] = df.index / FPS

    # Plotting
    plt.figure(figsize=(15, 6))  # Increased figure width
    if show_both_eyes:
        plt.plot(df['time'], df[0], label='Eye 0')
        plt.plot(df['time'], df[1], label='Eye 1')
    if show_mean:
        plt.plot(df['time'], df['mean_diameter'], label='Mean Diameter', linestyle='-')


    plt.title('Eye Diameters Over Time')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Diameter')
    plt.legend()
    plt.show()
    
def plot_cumulative_heatmap_of_gaze(start_index, end_index):
    plt.clf()  # Clear the current figure
    start_index = min(start_index,end_index-2)
    current_data = gaze_positions_df.iloc[start_index:end_index]
    image_size = [1920, 1080]  # Example image size (width, height)
    x = current_data['norm_pos_x'] * image_size[0]
    y = current_data['norm_pos_y'] * image_size[1]
    kde = gaussian_kde([x, y], bw_method='silverman')

    x_grid, y_grid = np.meshgrid(np.linspace(0, image_size[0], 100), np.linspace(0, image_size[1], 100))
    grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()])

    z = kde(grid_points).reshape(x_grid.shape)

    plt.figure(figsize=(12, 6))
    plt.imshow(z, origin='lower', aspect='auto', extent=[0, image_size[0], 0, image_size[1]], cmap='hot')
    plt.colorbar(label='Density')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('Gaze Position Heatmap Without Headpose')

def plot_headpose_and_gaze(ax, translated_rect, markers, gaze_position, eye_diameter):

    for j in range(4):
        start_point = translated_rect[j]
        end_point = translated_rect[(j + 1) % 4]
        # Adjusted line width
        ax.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color=FOV_COLOR, linewidth=FOV_THICKNESS * 2)

    # Draw Markers
    for marker_id, marker_center, marker_corners in markers:
        for j in range(4):
            start_point = marker_corners[j]
            end_point = marker_corners[(j + 1) % 4]
            # Adjusted line width
            ax.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color=MARKER_VISIBLE_COLOR, linewidth=MARKER_THICKNESS * 2)

    # Draw Gaze and Dilation Circle
    eye_dilation_radius = int(eye_diameter / EYE_DIAMETER_NORMALIZING_FACTOR * GAZE_CIRCLE_RADIUS)
    # Adjusted circle radius
    circle1 = plt.Circle((gaze_position[0], gaze_position[1]), GAZE_CIRCLE_RADIUS * 2, color=GAZE_CIRCLE_COLOR, fill=False)
    circle2 = plt.Circle((gaze_position[0], gaze_position[1]), eye_dilation_radius * 2, color=EYE_DILATION_COLOR, fill=False)
    ax.add_artist(circle1)
    ax.add_artist(circle2)

    # Set limits and aspect ratio
    ax.set_xlim(0, IMG_SIZE[0])
    ax.set_ylim(IMG_SIZE[1], 0)

def plot_eye_diameters(ax, index, show_eyes, show_mean, range_around_index=100):
    # Calculate start and end indices
    start_index = max(0, index - range_around_index)
    end_index = min(len(pupil_dilations_df), index + range_around_index + 1)
    df = pupil_dilations_df.iloc[start_index:end_index]

    # Calculate the mean of eye diameters
    df['mean_diameter'] = pupil_dilations_df.mean(axis=1)

    # Convert frame index to seconds
    df['time'] = df.index / FPS

    if show_eyes:
        ax.plot(df['time'], df[0], label='Eye 0')
        ax.plot(df['time'], df[1], label='Eye 1')
    if show_mean:
        ax.plot(df['time'], df['mean_diameter'], label='Mean Diameter', linestyle='-')

    ax.axvline(x=index / FPS, color='red', linestyle='--')  # Mark the specified index
    ax.set_title(f'Eye Diameters around second {index/FPS:.2f}')
    ax.set_xlabel('Seconds')
    ax.set_ylabel('Eye Diameter')
    ax.legend()
    ax.grid(True)

def plot_markers_over_time(ax, marker_counts, index, range_around_index=100):
    
    start_index = max(0, index - range_around_index)
    end_index = min(len(marker_counts), index + range_around_index + 1)
    
    frame_numbers = list(range(start_index, end_index))
    marker_values = marker_counts[start_index:end_index]
    
    ax.plot(frame_numbers, marker_values, linestyle='-', label='Markers over time')
    ax.axvline(x=index, color='red', linestyle='--')
    ax.set_title(f'Markers Count around second {index/ FPS:.2f}')
    ax.set_xlabel('Seconds')
    ax.set_ylabel('Number of Markers')
    ax.set_ylim(-0.5, 8.5)  # Adjust y-axis limits

    ax.legend()
    ax.grid(True)

def plot_blinks(ax, continous_blinks, index, range_around_index=300):
    # Calculate the frame range to display
    start_index = max(0, index - range_around_index)
    end_index = min(number_of_frames, index + range_around_index + 1)

    # Create a list for x-axis (frame number) and y-axis (blink presence)
    frame_numbers = list(range(start_index, end_index))
    continous_blinks = continous_blinks[start_index:end_index]
    ax.plot(frame_numbers, continous_blinks, linestyle='-', label='Blinks over time')
    ax.axvline(x=index, color='red', linestyle='--')  # Mark the specified index
    ax.set_title(f'Blinks around second {index/ FPS:.2f}')
    ax.set_xlabel('Seconds')
    ax.set_ylabel('Eye Diameter')
    ax.set_ylim(-0.5, 1.5)  # Corrected y-axis limits setting

    ax.legend()
    ax.grid(True)


def plot_gaze_heatmap(ax, head_poses, gaze_positions, frame_index, range_of_frames):
    # Assuming the input head_poses and gaze_positions are available as global variables

    # Get the heatmap gaze positions using your function
    heatmap_gaze_positions = get_heatmap_gaze_positions(gaze_positions, head_poses, frame_index, range_of_frames )

    # Convert the heatmap gaze positions to x and y arrays
    if len(heatmap_gaze_positions):
        x, y = zip(*heatmap_gaze_positions)
    else:
        x,y = 0,0

    # Perform Kernel Density Estimation
    kde = gaussian_kde([x, y], bw_method='silverman')

    # Create a grid of points for evaluation
    x_grid, y_grid = np.meshgrid(np.linspace(min(x), max(x), 100), np.linspace(min(y), max(y), 100))
    grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()])

    # Evaluate the KDE on the grid
    z = kde(grid_points).reshape(x_grid.shape)

    # Plot the heatmap
    im = ax.imshow(z, origin='lower', aspect='auto', extent=[min(x), max(x), min(y), max(y)], cmap='hot')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    ax.set_title('Gaze Position Heatmap')

    return im


## Dashboard

### Dashboard Main Visualization Function

In [9]:


def visualize(frame_index, blink_number, dilation_number, show_both_eyes_dilation, show_mean_dilation, marker_count_number, heatmap_number):
    gaze_position = gaze_positions_df.iloc[frame_index]
    pitch, yaw, roll = head_poses_df.iloc[frame_index]
    markers = marker_positions[frame_index]
    eye_diameter = pupil_dilations_df.iloc[frame_index]
    frame = frames[frame_index]

    fig, axs = plt.subplots(3, 2, figsize=(16, 16))


    fov_center, fov_rectangle = get_translated_view_rectangle(pitch, -roll, yaw)
    adjusted_gaze_pos = estimate_gaze_pos(fov_center, gaze_position, roll)
    head_pose = {"yaw": yaw, "roll": roll, "pitch": pitch}
    diameter_mean = (eye_diameter[0] + eye_diameter[1]) / 2
    adjusted_normalized_markers = get_adjusted_normalized_markers(fov_center,markers,roll)

    plot_headpose_and_gaze(axs[0,1],fov_rectangle,adjusted_normalized_markers, adjusted_gaze_pos, diameter_mean)

    plot_eye_diameters(axs[1,0],frame_index, show_both_eyes_dilation, show_mean_dilation, dilation_number)

    plot_blinks(axs[1,1],continous_blinks_list,frame_index,blink_number)

    plot_markers_over_time(axs[2,0],marker_counts_per_frame, frame_index, marker_count_number)

    im = plot_gaze_heatmap(axs[2,1],head_poses_df,gaze_positions_df,frame_index,heatmap_number)
    cbar = fig.colorbar(im, ax=axs[2,1])
    cbar.set_label('Density')

    axs[0,0].imshow(frame)  # Display the actual frame
    plt.tight_layout()
    plt.show()
    
slider_layout = Layout(width='500px')  # Adjust the width of the slider
description_layout = Layout(width='200px')  # Adjust the width of the description



### Configuration widgets:
In here, configuration widgets such as sliders and checks are declared and initiliazed. 

* Change continous-update to false, if it gets too laggy. It will render only when you release the slider then.

In [10]:
continuous_update = True


cumulative_heatmap_start_index = widgets.IntSlider(
    value=0, min=0, max=len(gaze_positions_df), step=1,
    description='Start Index:', continuous_update=False
)

cumulative_heatmap_end_index = widgets.IntSlider(
    value=len(gaze_positions_df), min=1, max=len(gaze_positions_df), step=1,
    description='End Index:', continuous_update=False
)

show_both_eyes_cumulative = widgets.Checkbox(
    value=False,
    description='Show left and right eye',
    disabled=False
)
show_mean_cumulative = widgets.Checkbox(
    value=True,
    description='Show mean diameter',
    disabled=False
)

frame_index_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=number_of_frames-1,
    step=1,
    description='Frame-Index:',
    continuous_update=continuous_update,
    layout=slider_layout,
    style={'description_width': description_layout.width}
)

number_of_frames_blinks_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=number_of_frames-1,
    step=1,
    description='Blink Frame Range:',
    continuous_update=continuous_update,
    layout=slider_layout,
    style={'description_width': description_layout.width}
)

number_of_frames_dilation_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=number_of_frames-1,
    step=1,
    description='Dilation Frame Range:',
    continuous_update=continuous_update,
    layout=slider_layout,
    style={'description_width': description_layout.width}
)
show_both_eyes_dilation = widgets.Checkbox(
    value=False,
    description='Show left and right eye',
    disabled=False
)
show_mean_dilation = widgets.Checkbox(
    value=True,
    description='Show mean diameter',
    disabled=False
)

number_of_frames_marker_count_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=number_of_frames-1,
    step=1,
    description='Marker Count Frame Range:',
    continuous_update=continuous_update,
    layout=slider_layout,
    style={'description_width': description_layout.width}
)

number_of_frames_heatmap_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=number_of_frames-1,
    step=1,
    description='Heatmap Frame Range:',
    continuous_update=continuous_update,
    layout=slider_layout,
    style={'description_width': description_layout.width}
)
checks_box = widgets.VBox([show_both_eyes_dilation,show_mean_dilation])

sliders_box = widgets.VBox([frame_index_slider,number_of_frames_blinks_slider,number_of_frames_dilation_slider,number_of_frames_heatmap_slider,number_of_frames_marker_count_slider])

control_box = widgets.HBox([sliders_box,checks_box])

### Dashboard Panel

In [11]:

interactive_plot = widgets.interactive(visualize, 
                    frame_index=frame_index_slider,
                    blink_number=number_of_frames_blinks_slider,
                    dilation_number=number_of_frames_dilation_slider,
                    show_both_eyes_dilation = show_both_eyes_dilation,
                    show_mean_dilation = show_mean_dilation,
                    marker_count_number=number_of_frames_marker_count_slider,
                    heatmap_number=number_of_frames_heatmap_slider)

interactive_output = interactive_plot.children[-1]

final_layout = widgets.VBox([control_box,interactive_output])

display(final_layout)


#### Cumulative Diagram Of Eye Diameters Over Time:
This is cumulative diagram of eye diameters over time. One can examine eye dilation through this diagram.

In [12]:
widgets.interactive(plot_cumulatively_eye_diameters, show_eyes=show_both_eyes_cumulative, show_mean=show_mean_cumulative)

interactive(children=(Checkbox(value=False, description='show_both_eyes'), Checkbox(value=True, description='S…

#### Cumulative Gaze-Position Heatmap:
This visualization can be used to estimate when user is looking on his POV. This estimation and visualization does not consider headpose changes, only visualize the estimated gaze from eye cameras.

In [13]:
widgets.interactive(plot_cumulative_heatmap_of_gaze, start_index=cumulative_heatmap_start_index, end_index=cumulative_heatmap_end_index)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Start Index:', max=170), IntSli…